In [9]:
import mediapipe
import cv2
import pickle
import numpy as np
from math import cos, sin

In [2]:


cap = cv2.VideoCapture(0)

while cap.isOpened():

  ret, frame = cap.read()

  faceModule = mediapipe.solutions.face_mesh

  with faceModule.FaceMesh(static_image_mode=True) as face:

    # processing the image to detect the face and then generating the land marks (468 for each x,y,z).

    results = face.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks != None:

      for face in results.multi_face_landmarks:

          # 

          for j,landmark in enumerate(face.landmark):

              x = landmark.x

              y = landmark.y

              shape = frame.shape 

              relative_x = int(x * shape[1])

              relative_y = int(y * shape[0])

              cv2.circle(frame, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)

              #cv2.putText(frame, str(j), (int(relative_x),int(relative_y)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 2)

  cv2.imshow('Raw Webcam Feed', frame)

  if cv2.waitKey(10) & 0xFF == ord('q'):

    break



cap.release()

cv2.destroyAllWindows()

In [3]:
def draw_axis(img, pitch,yaw,roll, tdx=None, tdy=None, size = 100):



    yaw = -yaw

    if tdx != None and tdy != None:

        tdx = tdx

        tdy = tdy

    else:

        height, width = img.shape[:2]

        tdx = width / 2

        tdy = height / 2



    # X-Axis (red)

    x1 = size * (cos(yaw) * cos(roll)) + tdx

    y1 = size * (cos(pitch) * sin(roll) + cos(roll) * sin(pitch) * sin(yaw)) + tdy



    # Y-Axis (green)

    x2 = size * (-cos(yaw) * sin(roll)) + tdx

    y2 = size * (cos(pitch) * cos(roll) - sin(pitch) * sin(yaw) * sin(roll)) + tdy



    # Z-Axis (blue)

    x3 = size * (sin(yaw)) + tdx

    y3 = size * (-cos(yaw) * sin(pitch)) + tdy



    cv2.line(img, (int(tdx), int(tdy)), (int(x1),int(y1)),(0,0,255),3)

    cv2.line(img, (int(tdx), int(tdy)), (int(x2),int(y2)),(0,255,0),3)

    cv2.line(img, (int(tdx), int(tdy)), (int(x3),int(y3)),(255,0,0),2)



    return img

In [5]:
with open('svr_yaw.pkl', 'rb') as f:

    yaw_model = pickle.load(f)

In [6]:
with open('svr_pitch.pkl', 'rb') as f:

    pitch_model = pickle.load(f)

In [7]:
with open('svr_roll.pkl', 'rb') as f:

    roll_model = pickle.load(f)

In [14]:
import cv2

cap = cv2.VideoCapture(0)

while cap.isOpened():

  ret, frame = cap.read()

  faceModule = mediapipe.solutions.face_mesh

  with faceModule.FaceMesh(static_image_mode=True) as face:

    # processing the image to detect the face and then generating the land marks (468 for each x,y,z).

    results = face.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks != None:

      X = []

      Y = []

      for face in results.multi_face_landmarks:

          # 

          for j,landmark in enumerate(face.landmark):

              x = landmark.x

              y = landmark.y

          

              shape = frame.shape 

              relative_x = int(x * shape[1])

              relative_y = int(y * shape[0])

#               cv2.circle(frame, (relative_x, relative_y), radius=1, color=(0, 255, 0), thickness=2)
              X.append(relative_x)

              Y.append(relative_y)



          X = np.array(X)

          Y = np.array(Y)



          # centering the points of x and y values arround the point no. 99

          X_center = X - X[99]

          Y_center = Y - Y[99]

          # computing the distance between the point 152 and 10

          d = np.linalg.norm(np.array((X[152],Y[152])) - np.array((X[10],Y[10])))

          # normalizing the points

          X_norm = X_center/d

          Y_norm = Y_center/d

          X_norm = X_norm

          Y_norm = Y_norm

          # concatinating the x and y points to predict the labels (pitch,yaw,roll)

          points = np.hstack([X_norm,Y_norm]).reshape(1,-1)

          # predicting the 3 angels to draw the axis on the image



          pitch_pred = pitch_model.predict(points)

          yaw_pred = yaw_model.predict(points)

          roll_pred = roll_model.predict(points)
            
          frame= draw_axis(frame,pitch_pred,yaw_pred,roll_pred)

          



              

  
  cv2.imshow('Raw Webcam Feed', frame)
    
  if cv2.waitKey(10) & 0xFF == ord('q'):

    break



cap.release()

cv2.destroyAllWindows()